# Project 4
Chen Liang

The target of this project is to detect 'punching wall or something'. Dataset used is HMDB. Since the output will be a probability of 'punching' or not, the model is not designed to detect all categories in this dataset.

In [1]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import random
from sklearn.utils import shuffle

## Data Preprocessing

### Select Files from Json

In [2]:
fpath='hmdb/'
punch_fnames=[join(fpath+'punch/', f) for f in listdir(fpath+'punch/') if isfile(join(fpath+'punch/', f))]
#notpunch_fnames=[join(fpath+'notpunch/', f) for f in listdir(fpath+'notpunch/') if isfile(join(fpath+'notpunch/', f))]
notpunch_fnames=[join(fpath+'mess/', f) for f in listdir(fpath+'mess/') if isfile(join(fpath+'mess/', f))]

### Generate Images from Video

In [3]:
from joblib import Parallel, delayed
from video_to_img_hmdb import video_to_img
import numpy as np

#### Punch

In [4]:
num_processes = 12
_=Parallel(n_jobs=num_processes)(delayed(video_to_img)\
                               (str(video_id),'hframes/punch/',is_merged=True,merge_size=5) for video_id in punch_fnames)

#### Not Punch
Randomly choose same amount of video from dfnothit

In [5]:
num_processes = 12
_=Parallel(n_jobs=num_processes)(delayed(video_to_img)\
                               (str(video_id),'hframes/notpunch/',is_merged=True,merge_size=5)\
                                 for video_id in notpunch_fnames)

### Load and Organize Data

<b>Note:</b> due to limited memory on laptop, only part of data is used for training

In [6]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np

In [7]:
frame_path='hframes/'
punch_imgs=[join(frame_path+'punch/', f) for f in listdir(frame_path+'punch/') if isfile(join(frame_path+'punch/', f))]
notpunch_imgs=[join(frame_path+'notpunch/', f) for f in listdir(frame_path+'notpunch/') if isfile(join(frame_path+'notpunch/', f))]

In [8]:
print('punch imgs: ',len(punch_imgs),'. not_punch imgs: ',len(notpunch_imgs))

punch imgs:  8921 . not_punch imgs:  9446


#### Train

In [9]:
random.randrange(2)  

0

In [10]:
img_w=224
img_h=224

In [11]:
Xtl=[]
for f in punch_imgs[500:]:
    Xtl.append(cv2.imread(f))
for f in notpunch_imgs[500:]:
    Xtl.append(cv2.imread(f))

In [12]:
yt=([1]*(len(punch_imgs)-500))+([0]*(len(notpunch_imgs)-500))
yt=np.array(yt)

In [13]:
Xtl, yt= shuffle(Xtl, yt)
Xt=np.stack(Xtl,axis=0)
del Xtl

In [14]:
# Xt=np.empty([8921+9446-1000,img_h,img_w,3],dtype='uint8')
# i=0
# p_i=500
# np_i=500
# for f in punch_imgs[500:]:
#     Xt[i,:,:,:]=cv2.imread(f)
#     i+=1
# for f in notpunch_imgs[500:]:
#     Xt[i,:,:,:]=cv2.imread(f)
#     i+=1

In [15]:
Xt.shape

(17367, 224, 224, 3)

In [16]:
Xvl=[]
xi=[]
for f in punch_imgs[:500]:
    Xvl.append(cv2.imread(f))
    xi.append(1)
for f in notpunch_imgs[:500]:
    Xvl.append(cv2.imread(f))
    xi.append(0)

In [17]:
yv=[1]*500+[0]*500
yv=np.array(yv)

In [18]:
Xvl, yv= shuffle(Xvl, yv)
Xv=np.stack(Xvl,axis=0)
del Xvl

In [31]:
cv2.imshow('qwe',Xt[17066,:,:,:])
cv2.waitKey(0)
cv2.destroyAllWindows()


## Model Training

In [19]:
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from scipy import stats
import sklearn.metrics as metrices
import seaborn as sns

In [20]:
input_shape=(224, 224, 3)

In [21]:
def create_model():
    model1=models.Sequential()
    model1.add(layers.Conv2D(64,kernel_size=(3,3),strides=(2,2),activation='relu',input_shape=input_shape))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Dropout(0.25))
    model1.add(layers.Flatten())
    model1.add(layers.Dense(128, activation='relu'))
    model1.add(layers.Dense(2, activation='softmax'))
    return model1

In [22]:
model=create_model()

In [23]:
from tensorflow.keras.utils import to_categorical
ytb = to_categorical(yt)
yvb = to_categorical(yv)

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(Xt, yt, epochs=30, 
                    validation_data=(Xv, yv))

Train on 17367 samples, validate on 1000 samples
Epoch 1/30
17367/17367 [==============================] - 17s 972us/sample - loss: 0.3412 - accuracy: 0.8742 - val_loss: 0.4889 - val_accuracy: 0.9080
Epoch 2/30
17367/17367 [==============================] - 13s 756us/sample - loss: 0.0887 - accuracy: 0.9638 - val_loss: 0.2288 - val_accuracy: 0.9030
Epoch 3/30
17367/17367 [==============================] - 13s 761us/sample - loss: 0.0945 - accuracy: 0.9658 - val_loss: 0.6219 - val_accuracy: 0.9080
Epoch 4/30
17367/17367 [==============================] - 13s 761us/sample - loss: 0.0616 - accuracy: 0.9774 - val_loss: 0.6422 - val_accuracy: 0.9080
Epoch 5/30
17367/17367 [==============================] - 13s 759us/sample - loss: 0.0676 - accuracy: 0.9764 - val_loss: 1.3412 - val_accuracy: 0.9080
Epoch 6/30
17367/17367 [==============================] - 13s 760us/sample - loss: 0.0351 - accuracy: 0.9876 - val_loss: 0.9226 - val_accuracy: 0.9080
Epoch 7/30
17367/17367 [=====================

## Save Model

In [25]:
model.save('hmdbmodel2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: hmdbmodel2\assets
